# Bose-Hubbard model in 1d
https://quspin.github.io/QuSpin/examples/user-basis_example2.html#user-basis-example2-label

In [3]:
#
import sys, os

os.environ["KMP_DUPLICATE_LIB_OK"] = (
    "True"  # uncomment this line if omp error occurs on OSX for python 3
)
os.environ["OMP_NUM_THREADS"] = "1"  # set number of OpenMP threads to run in parallel
os.environ["MKL_NUM_THREADS"] = "1"  # set number of MKL threads to run in parallel
#

#
from quspin.operators import hamiltonian  # Hamiltonians and operators
from quspin.basis import boson_basis_1d  # Hilbert space spin basis_1d
from quspin.basis.user import user_basis  # Hilbert space user basis
from quspin.basis.user import (
    next_state_sig_32,
    op_sig_32,
    map_sig_32,
    count_particles_sig_32,
)  # user basis data types signatures
from numba import carray, cfunc  # numba helper functions
from numba import uint32, int32, float64  # numba data types
import numpy as np
from scipy.special import comb

import matplotlib.pyplot as plt # matplotlib

In [4]:
from quspin.operators import hamiltonian, exp_op  # Hamiltonians and operators
from quspin.basis import spinless_fermion_basis_1d  # Hilbert space fermion basis
from quspin.tools.block_tools import block_diag_hamiltonian  # block diagonalisation
import numpy as np  # generic math functions
import matplotlib.pyplot as plt  # plotting library

try:  # import python 3 zip function in python 2 and pass if already using python 3
    import itertools.izip as zip
except ImportError:
    pass

from quspin.basis import boson_basis_1d  # Hilbert space spin basis_1d

In [ ]:
def mathematica_export(name,array):
    f = open(name+".wl", "w")
    f.write(str(array).replace('[', '{').replace(']', '}'))
    f.close()

In [6]:
def energy(N,Np,U):
    # N : lattice sites
    # Np: total number of bosons
    J = 1.0  # uniform hopping
    deltaJ = -0.1  # bond dimerisation
    sps = 2  # states per site
    #
    ############   create same boson basis_1d object   #############
    basis_1d = boson_basis_1d(N, Nb=Np, sps=sps, kblock=0, pblock=1)
    #
    ############   create Hamiltonians   #############
    #
    J = -1.0
    # U = +1.0
    #
    hopping = [[-J - deltaJ * (-1) ** i, i, (i + 1) % N] for i in range(N)]
    int_bb = [[0.5 * U, j, j] for j in range(N)]
    int_b = [[-0.5 * U, j] for j in range(N)]
    #
    static = [["-+", hopping], ["-+", hopping], ["nn", int_bb], ["n", int_b]]
    dynamic = []
    #
    no_checks = dict(check_symm=False, check_pcon=False, check_herm=False)
    H_1d = hamiltonian(static, [], basis=basis_1d, dtype=np.float64, **no_checks)
    
    # construct operator n_1 = $n_{j=0}$
    n_1_static = [["n", [[1.0, 0]]]]
    n_1 = hamiltonian(
        n_1_static, [], basis=basis_1d, dtype=np.float64, check_herm=False, check_pcon=False
    )
    # construct operator n_2 = $n_{j=L/2}$
    n_2_static = [["n", [[1.0, N // 2]]]]
    n_2 = hamiltonian(
        n_2_static, [], basis=basis_1d, dtype=np.float64, check_herm=False, check_pcon=False
    )

    E, V = H_1d.eigh()
    
    Emin, Emax = H_1d.eigsh(k=2, which="BE", maxiter=1e4, return_eigenvectors=False)

    return Emin # n_1.expt_value(V)

In [7]:
energy(10,2,1.)

/var/folders/lg/kgsspcl94s11sjhvr12rf2tm0000gn/T/ipykernel_56227/3239471982.py:1: UserWarning: The following static operator strings do not obey T symm: ['n']
  energy(10,2,1.)


TypeError: Hamiltonian does not obey T symm! To turn off check, use check_symm=False in hamiltonian.

In [27]:
def gap(N,U): 
    #
    # N : lattice sites
    sps = 4  # states per site
    Np = N  # total number of bosons
    #
    ############   create same boson basis_1d object   #############
    basis_1d = boson_basis_1d(N, Nb=Np, sps=sps, kblock=0, pblock=1)
    #
    ############   create Hamiltonians   #############
    #
    J = -1.0
    # U = +1.0
    #
    hopping = [[+J, j, (j + 1) % N] for j in range(N)]
    int_bb = [[0.5 * U, j, j] for j in range(N)]
    int_b = [[-0.5 * U, j] for j in range(N)]
    #
    static = [["+-", hopping], ["-+", hopping], ["nn", int_bb], ["n", int_b]]
    dynamic = []
    #
    no_checks = dict(check_symm=False, check_pcon=False, check_herm=False)
    H_1d = hamiltonian(static, [], basis=basis_1d, dtype=np.float64, **no_checks)

    eigvalsh = H_1d.eigvalsh()

    return eigvalsh[1]-eigvalsh[0]

In [12]:
gap(8,1.)

np.float64(2.652303334292201)